In [ ]:
from evaluate import Evaluator

rp = "results/mistral_7b_no_src_ref_stratified_wmt22_ende_3200.pkl"
eva_ende = Evaluator("data/wmt22_mqm_en-de_3200.pkl", rp)
eva_zhen = Evaluator("data/wmt22_mqm_zh-en_3200.pkl", rp.replace("ende", "zhen"))

In [ ]:
print("ende")
ende_f1 = eva_ende.sf1_mf1("en")
print("zhen")
zhen_f1 = eva_zhen.sf1_mf1()
sum_num = [i + j for i, j in zip(ende_f1, zhen_f1)]
num_intersection, num_predict, num_label, num_major_intersection, num_major_predict, num_major = sum_num

span_precision = num_intersection / num_predict
span_recall = num_intersection / num_label
sf1 = (2 * num_intersection) / (num_predict + num_label)
major_precision = num_major_intersection / num_major_predict
major_recall = num_major_intersection / num_major
mf1 = (2 * num_major_intersection) / (num_major_predict + num_major)

print("\navg")
print(f"SP: {num_intersection:>5} / {num_predict:>5} = {span_precision:.3f}")
print(f"SR: {num_intersection:>5} / {num_label:>5} = {span_recall:.3f}")
print(f"S-F1: {sf1:.3f}")
print(f"MP: {num_major_intersection:>5} / {num_major_predict:>5} = {major_precision:.3f}")
print(f"MR: {num_major_intersection:>5} / {num_major:>5} = {major_recall:.3f}")
print(f"M-F1: {mf1:.3f}")

In [ ]:
print("ende")
ende_mcc = eva_ende.mcc("de")
print("zhen")
zhen_mcc = eva_zhen.mcc()
sum_num = [i + j for i, j in zip(ende_mcc, zhen_mcc)]
tp, tn, fp, fn = sum_num
mcc_numerator = (tp * tn) - (fp * fn)
mcc_denominator = ((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)) ** 0.5
mcc = mcc_numerator / mcc_denominator

print("\navg")
print(f"MCC tp: {tp}")
print(f"MCC tn: {tn}")
print(f"MCC fp: {fp}")
print(f"MCC fn: {fn}")
print(f"MCC: {mcc:.3f}")

In [ ]:
print("ende")
ende_confusion_dict = eva_ende.category_prf1()
print("zhen")
zhen_confusion_dict = eva_zhen.category_prf1()
merged_dict = {
    k: [v1 + v2 for v1, v2 in zip(v_ende, v_zhen)] 
    for k, v_ende, v_zhen in zip(
        ende_confusion_dict.keys(), ende_confusion_dict.values(), zhen_confusion_dict.values()
    )
}

for k, v in merged_dict.items():
    precision = v[0] / (v[0] + v[2]) if v[0] > 0 else 0
    recall = v[0] / (v[0] + v[1]) if v[0] > 0 else 0
    f1 = 2 * v[0] / (2 * v[0] + v[1] + v[2]) if v[0] > 0 else 0
    print(f"{k}: P: {precision} R: {recall} F1: {f1} num: {v[3]}")
print(merged_dict)